In [1]:
import sys
sys.path

['c:\\coding\\QuantFreedom\\nax tests',
 'c:\\Users\\New User\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'c:\\Users\\New User\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'c:\\Users\\New User\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'c:\\Users\\New User\\AppData\\Local\\Programs\\Python\\Python310',
 'c:\\coding\\qfFree',
 '',
 'c:\\coding\\qfFree\\lib\\site-packages',
 'c:\\coding\\quantfreedom',
 'c:\\coding\\qfFree\\lib\\site-packages\\win32',
 'c:\\coding\\qfFree\\lib\\site-packages\\win32\\lib',
 'c:\\coding\\qfFree\\lib\\site-packages\\Pythonwin']